#### Model Training

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.preprocessing import OneHotEncoder

In [9]:
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

#### 1. Dropping unneccasary columns & filling null values

In [10]:
train_data = train_data.drop(["Unnamed: 0"],axis=1)
test_data = test_data.drop(["Unnamed: 0"], axis=1)

In [11]:
train_data.sample(4)

,acc_max,gyro_max,acc_kurtosis,gyro_kurtosis,label,lin_max,acc_skewness,gyro_skewness,post_gyro_max,post_lin_max,fall
917,20.157368,3.628130,1.315046,-0.116667,WAL,5.158297,0.640823,0.718020,-0.968422,-0.114050,0
378,22.929192,6.603116,9.972329,14.063034,FKL,8.395728,1.975307,3.307560,6.525168,8.272332,1
776,24.993778,11.631272,-1.175077,2.450054,JOG,12.681553,0.035853,1.469936,0.056627,0.554723,0
1255,21.900704,3.872433,0.492031,0.122949,STU,8.304420,0.120527,0.714288,-0.675202,1.245424,0


In [12]:
test_data.sample(4)

,acc_max,gyro_max,acc_kurtosis,gyro_kurtosis,label,lin_max,acc_skewness,gyro_skewness,post_gyro_max,post_lin_max,fall
273,24.741495,7.490672,-1.259162,6.526817,JUM,12.010318,0.558980,1.873410,-1.041825,1.686502,0
288,22.113377,2.845409,-1.429069,3.797065,JUM,6.795975,0.528236,1.488655,-0.868545,-1.686062,0
322,9.930902,0.281440,2.807073,-0.994296,STD,0.209536,-0.220630,0.122732,0.051937,0.134645,0
195,13.962384,2.747275,4.815797,-0.845245,CSO,2.505607,0.799709,0.499626,-0.049347,-0.238592,0


In [13]:
train_data.isnull().sum()

acc_max          0
gyro_max         0
acc_kurtosis     0
gyro_kurtosis    0
label            0
lin_max          0
acc_skewness     0
gyro_skewness    0
post_gyro_max    0
post_lin_max     0
fall             0
dtype: int64

In [14]:
test_data.isnull().sum()

acc_max          0
gyro_max         0
acc_kurtosis     0
gyro_kurtosis    0
label            0
lin_max          0
acc_skewness     0
gyro_skewness    0
post_gyro_max    0
post_lin_max     0
fall             0
dtype: int64

#### 3. Splitting independent and dependent feature for training and testing set

In [25]:
X_train = train_data.drop(["fall"], axis=1)
X_test = test_data.drop(["fall"], axis=1)

y_train = train_data[["fall"]]
y_test = test_data[["fall"]]

#### 4. Encoding parts

In [33]:
categorical_cols = X_train.select_dtypes(include="object").columns
numerical_cols = X_train.select_dtypes(exclude="object").columns

In [32]:
categorical_cols

Index(['label'], dtype='object')

In [34]:
numerical_cols

Index(['acc_max', 'gyro_max', 'acc_kurtosis', 'gyro_kurtosis', 'lin_max',
       'acc_skewness', 'gyro_skewness', 'post_gyro_max', 'post_lin_max'],
      dtype='object')

#### 5. Defining the values for categorical columns

In [35]:
categorical_cols

Index(['label'], dtype='object')

In [38]:
for i in categorical_cols:
    print(i, train_data[i].unique())

label ['SDL' 'FOL' 'FKL' 'BSC' 'STU' 'JUM' 'STD' 'WAL' 'CSI' 'CSO' 'STN' 'JOG'
 'SCH']


#### Libraries for handlind missing values, feature scaling and feature engineering

In [39]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [40]:
train_data.columns

Index(['acc_max', 'gyro_max', 'acc_kurtosis', 'gyro_kurtosis', 'label',
       'lin_max', 'acc_skewness', 'gyro_skewness', 'post_gyro_max',
       'post_lin_max', 'fall'],
      dtype='object')

#### 6. Feature Splittings

In [52]:
numerical_features = ["acc_max", "gyro_max", "acc_kurtosis", "gyro_kurtosis",
                      "lin_max", "acc_skewness", "gyro_skewness", "post_gyro_max",
                      "post_lin_max"]

one_hot_features = ["label"]

#### 7. Defining the transformers

In [53]:
numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

#### 8. We created numerical separately and categorical separately, now we need to combine this

In [54]:
preprocessor = ColumnTransformer(
    transformers = 
    [
    ("oh", oh_transformer, one_hot_features),
    ("num", numeric_transformer, numerical_features)
    ]
)

In [55]:
X_train.head()

,acc_max,gyro_max,acc_kurtosis,gyro_kurtosis,label,lin_max,acc_skewness,gyro_skewness,post_gyro_max,post_lin_max
0,26.039919,7.309797,20.378162,2.782476,SDL,11.131080,3.891361,1.592927,7.086618,10.790400
1,25.864500,6.511954,14.187190,5.324864,FOL,7.945561,3.022175,2.376939,6.325522,7.719352
2,27.524501,12.944099,31.855926,22.891186,FOL,14.454818,4.849024,4.283890,12.888111,14.368784
3,30.647705,11.694868,23.608764,9.287735,FOL,15.228303,3.921537,2.794609,11.549971,14.944151
4,26.373917,11.168424,14.318453,15.983202,FKL,10.007396,3.087975,3.363557,11.057636,9.753058


In [56]:
X_test.head()

,acc_max,gyro_max,acc_kurtosis,gyro_kurtosis,label,lin_max,acc_skewness,gyro_skewness,post_gyro_max,post_lin_max
0,28.055199,10.794617,21.334536,34.163811,FOL,13.880578,3.283404,4.577283,10.755339,13.762561
1,26.639681,8.785024,13.518671,12.812894,FOL,15.789372,3.301849,3.464729,8.277714,15.341656
2,25.045219,5.307413,21.603060,4.754182,FOL,11.592445,3.124714,2.268676,4.976134,11.303823
3,24.102184,8.929061,24.647657,18.595684,FOL,10.107835,4.522305,3.955288,8.719755,9.727437
4,31.668808,10.714750,18.008912,15.086251,FOL,14.138265,3.142132,3.530621,10.580791,13.935016


#### 9. Applying the preprocessor code

In [57]:
preprocessor.fit_transform(X_train)

array([[ 0.        ,  0.        ,  0.        , ..., -0.0331466 ,
         1.12315231,  1.1222695 ],
       [ 0.        ,  0.        ,  0.        , ...,  0.75144965,
         0.90134963,  0.50684513],
       [ 0.        ,  0.        ,  0.        , ...,  2.65982166,
         2.81385571,  1.83936174],
       ...,
       [ 0.        ,  1.        ,  0.        , ..., -1.20220513,
        -0.39979802, -0.8673627 ],
       [ 0.        ,  0.        ,  0.        , ..., -1.31585641,
        -0.87123922, -0.84067464],
       [ 0.        ,  0.        ,  0.        , ..., -0.45591744,
        -0.99970136, -1.04422772]])

#### 10. Converting into dataframe

In [58]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train), columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test), columns=preprocessor.get_feature_names_out())

In [59]:
X_train

,oh__label_BSC,oh__label_CSI,oh__label_CSO,oh__label_FKL,oh__label_FOL,oh__label_JOG,oh__label_JUM,oh__label_SCH,oh__label_SDL,oh__label_STD,...,oh__label_WAL,num__acc_max,num__gyro_max,num__acc_kurtosis,num__gyro_kurtosis,num__lin_max,num__acc_skewness,num__gyro_skewness,num__post_gyro_max,num__post_lin_max
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.782516,0.761801,0.868992,-0.204544,0.752368,1.424829,-0.033147,1.123152,1.122269
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.750493,0.492665,0.352369,0.258236,0.002519,0.856668,0.751450,0.901350,0.506845
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.053532,2.662417,1.826786,3.455756,1.534752,2.050824,2.659822,2.813856,1.839362
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.623683,2.241015,1.138579,0.979580,1.716825,1.444554,1.169432,2.423887,1.954663
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.843489,2.063430,0.363323,2.198326,0.487860,0.899680,1.738804,2.280409,0.914391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,-2.146124,-1.690571,-0.815369,-0.706871,-1.855320,-1.116735,-1.494185,-0.941656,-1.039555
1424,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.760243,2.123944,-0.914700,-0.594901,2.109931,-0.988564,-0.818006,0.044603,-0.694420
1425,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.762464,-1.001107,-0.685854,-0.870155,-1.563703,-1.319585,-1.202205,-0.399798,-0.867363
1426,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.305571,-0.094915,-0.829789,-0.908994,-0.343341,-0.893406,-1.315856,-0.871239,-0.840675


In [60]:
X_test

,oh__label_BSC,oh__label_CSI,oh__label_CSO,oh__label_FKL,oh__label_FOL,oh__label_JOG,oh__label_JUM,oh__label_SCH,oh__label_SDL,oh__label_STD,...,oh__label_WAL,num__acc_max,num__gyro_max,num__acc_kurtosis,num__gyro_kurtosis,num__lin_max,num__acc_skewness,num__gyro_skewness,num__post_gyro_max,num__post_lin_max
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.150412,1.937334,0.948800,5.507663,1.399580,1.027426,2.953433,2.192311,1.717877
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.892005,1.259439,0.296583,1.621249,1.848897,1.039483,1.840051,1.470268,2.034321
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.600931,0.086338,0.971207,0.154357,0.860970,0.923695,0.643106,0.508103,1.225157
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.428777,1.308027,1.225272,2.673865,0.511503,1.837258,2.330975,1.599090,0.909256
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.810089,1.910392,0.671284,2.035058,1.460238,0.935080,1.905992,2.141444,1.752437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,-0.158907,-0.790509,-0.612922,-0.471549,-0.361401,-0.344906,-0.603491,-1.168897,-0.678924
352,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.063350,-0.547659,-0.648110,-0.634417,-0.246815,-0.326138,-0.717211,-0.837620,-0.643454
353,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.035945,-0.316548,-0.732627,-0.817288,-0.535514,-0.566645,-1.222507,-1.002456,-0.810833
354,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.502175,0.328938,-0.886916,-0.813516,0.826015,-0.945854,-1.313887,-0.633723,-0.160778


#### 11. Model Training

In [61]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

from sklearn.metrics import accuracy_score, classification_report,ConfusionMatrixDisplay,precision_score, recall_score, f1_score

In the real world scenario we are not going to train the one model, we are going to train the multiple models.

In [62]:
models ={
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Support Vector Classifier" : SVC(),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Ada Boosting": AdaBoostClassifier(),
    "K-Neighbours" : KNeighborsClassifier()
}

In [79]:
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    ## Make Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Training set performance
    model_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
    model_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score
    model_train_precision = precision_score(y_train, y_train_pred) # Calculate Precision
    model_train_recall = recall_score(y_train, y_train_pred) # Calculate Recall


    # Test set performance
    model_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
    model_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score
    model_test_precision = precision_score(y_test, y_test_pred) # Calculate Precision
    model_test_recall = recall_score(y_test, y_test_pred) # Calculate Recall


    print(list(models.keys())[i])

    print('Model performance for Training set')
    print("- Accuracy: {:.4f}".format(model_train_accuracy))
    print('- F1 score: {:.4f}'.format(model_train_f1))
    print('- Precision: {:.4f}'.format(model_train_precision))
    print('- Recall: {:.4f}'.format(model_train_recall))
    
    print('----------------------------------')
    
    print('Model performance for Test set')
    print('- Accuracy: {:.4f}'.format(model_test_accuracy))
    print('- F1 score: {:.4f}'.format(model_test_f1))
    print('- Precision: {:.4f}'.format(model_test_precision))
    print('- Recall: {:.4f}'.format(model_test_recall))
    
    print('='*35)
    print('\n')

C:\Users\msant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\msant\AppData\Local\Temp\ipykernel_4468\1104985485.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


Logistic Regression
Model performance for Training set
- Accuracy: 0.9993
- F1 score: 0.9993
- Precision: 0.9984
- Recall: 1.0000
----------------------------------
Model performance for Test set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000


Decision Tree
Model performance for Training set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
----------------------------------
Model performance for Test set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000


Random Forest
Model performance for Training set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
----------------------------------
Model performance for Test set
- Accuracy: 0.9972
- F1 score: 0.9972
- Precision: 1.0000
- Recall: 0.9935


Support Vector Classifier
Model performance for Training set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
----------------------------------
Model performance for 

C:\Users\msant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\msant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\msant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of

Gradient Boosting
Model performance for Training set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
----------------------------------
Model performance for Test set
- Accuracy: 0.9972
- F1 score: 0.9972
- Precision: 1.0000
- Recall: 0.9935




C:\Users\msant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Ada Boosting
Model performance for Training set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
----------------------------------
Model performance for Test set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000




C:\Users\msant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


K-Neighbours
Model performance for Training set
- Accuracy: 0.9979
- F1 score: 0.9979
- Precision: 0.9951
- Recall: 1.0000
----------------------------------
Model performance for Test set
- Accuracy: 0.9972
- F1 score: 0.9972
- Precision: 0.9935
- Recall: 1.0000




In [80]:
list(models.keys())

['Logistic Regression',
 'Decision Tree',
 'Random Forest',
 'Support Vector Classifier',
 'Naive Bayes',
 'Gradient Boosting',
 'Ada Boosting',
 'K-Neighbours']

                                                        Completed